In [53]:
import pickle
import re
import numpy as np

### functions

def get_start_pos(filename):
    """
    File names are structured [level]_[start x pos]_[start y pos].pkl
    Returns startin positions from file name
    """
    matches = re.findall(r'\d+', filename)# Use regular expression to find all groups of digits
    return map(int, matches[1:3])# Extract and return the second and third numbers

def apply_action(state, action):
    """
    Move the ball up, down, left, or right
    There are faster ways to do this, but this is a good general use case
    """
    def move_press(grid, xp, yp, dx, dy):
        while 0 <= xp < len(grid) and 0 <= yp < len(grid) and not grid[xp][yp].block:
            grid[xp][yp].set_active(True)
            xp += dx
            yp += dy
        return [grid, xp - dx, yp - dy]
    
    f_mapping = {'U': (move_press, -1, 0), 'D': (move_press, 1, 0), 'L': (move_press, 0, -1), 'R': (move_press, 0, 1)}
    
    xp, yp, grid = state.x_pos, state.y_pos, state.grid
    
    move_function, dx, dy = f_mapping[action]
    state.grid, state.x_pos, state.y_pos = move_function(grid, xp, yp, dx, dy)

def is_end_state(state):
    #Check if the game is done
    return np.sum([not cell.active and not cell.block for row in state.grid for cell in row]) == 0

###### Cell Class

class Cell:
    """
    Cell classes describe the the cells in the grid
    They can either be blocked (a wall) or not (traversable cell)
    If not blocked they can be active (stepped on) or inactive (not traversed)
    """
    def __init__(self, block, active):
        if isinstance(block, bool) and isinstance(active, bool):
            self.block = block
            self.active = active
        else:
            raise ValueError("Active and Block must be a boolean")

    def set_block(self, block):
        if not isinstance(block, bool):
            raise ValueError("Block must be a boolean")
        self.block = block

    def set_active(self, active):
        if not isinstance(active, bool):
            raise ValueError("Active must be a boolean")
        self.active = active
    

###### Game state class

class Game_state:
    """
    Load and save the grid. Contains x_pos of the ball, y_pos of the ball
    Grid size, blocked cells, and active cells within the grid
    """
    def __init__(self, x_pos, y_pos, grid):
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.grid = grid
        
    def save_grid(self, fn):
        filename = fn+".pkl"
        if filename:
            with open(filename, 'wb') as file:
                pickle.dump((len(self.grid), self.grid), file)

    def load_grid(self, filename):
        if filename:
            with open(filename, 'rb') as file:
                size, loaded_grid = pickle.load(file)
                self.grid = loaded_grid
                self.x_pos, self.y_pos = get_start_pos(filename)

In [62]:
init_state = Game_state(0, 0, 0) #initial state
init_state.load_grid('LEVELS\\11_8_0.pkl') #Load level 11

print('--- Empty grid layout ---\n\n')
print("Start Position:")
print(init_state.y_pos, init_state.x_pos)
print()
for row in init_state.grid:
    for cell in row:
        print(1 if cell.block else 0, end=' ')
    print()

--- Empty grid layout ---


Start Position:
0 8

0 0 0 1 0 0 0 1 1 
0 1 0 1 0 1 0 1 1 
0 1 0 1 0 1 0 1 1 
0 0 0 0 0 0 0 1 1 
1 1 0 1 0 1 1 1 1 
0 0 0 0 0 0 0 1 1 
0 1 0 1 0 1 0 1 1 
0 1 0 1 0 1 0 1 1 
0 0 0 1 0 0 0 1 1 


In [63]:
print('\n--- Active cell layout ---\n\n') #should all be 0
for row in init_state.grid:
    for cell in row:
        print(2 if cell.active else 0, end=' ')
    print()


--- Active cell layout ---


0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 


In [64]:
apply_action(init_state, 'U') #Actions can be done ['U', 'D', 'L', 'R']
print('\n--- Whole Grid after first move---\n\n') #active cells = 2, blocked cells = 1, inactive cells = 0
print("New Position:")
print(init_state.y_pos, init_state.x_pos)
print()
for row in init_state.grid:
    for cell in row:
        print(2 if cell.active else (1 if cell.block else 0), end=' ')
    print()


--- Whole Grid after first move---


New Position:
0 5

0 0 0 1 0 0 0 1 1 
0 1 0 1 0 1 0 1 1 
0 1 0 1 0 1 0 1 1 
0 0 0 0 0 0 0 1 1 
1 1 0 1 0 1 1 1 1 
2 0 0 0 0 0 0 1 1 
2 1 0 1 0 1 0 1 1 
2 1 0 1 0 1 0 1 1 
2 0 0 1 0 0 0 1 1 


In [65]:
print('play level 1\n')

init_state = Game_state(0, 0, 0) #initial state
init_state.load_grid('LEVELS\\1_4_0.pkl') #Load level 1

for action in ['U', 'R', 'D', 'L']:
    apply_action(init_state, action)
    if is_end_state(init_state):
        print('You won!')
    else:
        print('Keep playing')

play level 1

Keep playing
Keep playing
Keep playing
You won!
